# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [18]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [19]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,-176.5597,73,100,1.79,NZ,1722818966
1,1,aral,40.5156,81.2636,81.2636,36,100,0.60,CN,1722818967
2,2,jamestown,42.0970,-79.2353,-79.2353,60,0,2.57,US,1722818968
3,3,kodiak,57.7900,-152.4072,-152.4072,100,100,4.63,US,1722818969
4,4,margate,51.3813,1.3862,1.3862,83,46,4.90,GB,1722818881


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [20]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', geo=True, tiles='OSM', size='Humidity', 
    color='City', hover_cols=['City', 'Humidity', 'Country'],
    title='City Humidity Map'
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_city_data_df = city_data_df[
    (city_data_df['Humidity'] < 60) & (city_data_df['Max Temp'] > 20)
]


# Drop any rows with null values
filtered_city_data_df = filtered_city_data_df.dropna()


# Display sample data
print(filtered_city_data_df.head())

    City_ID        City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
1         1        aral  40.5156   81.2636   81.2636        36         100   
9         9     udachny  66.4167  112.4000  112.4000        44         100   
25       25  solnechnyy  50.7214  136.6319  136.6319        57         100   
58       58     susuman  62.7833  148.1667  148.1667        38           4   
61       61    motygino  58.1831   94.7592   94.7592        56          12   

    Wind Speed Country        Date  
1         0.60      CN  1722818967  
9         4.81      RU  1722818977  
25        3.19      RU  1722818999  
58        2.55      RU  1722819039  
61        1.27      RU  1722819042  


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
print(hotel_df.head())

          City Country      Lat       Lng  Humidity Hotel Name
1         aral      CN  40.5156   81.2636        36           
9      udachny      RU  66.4167  112.4000        44           
25  solnechnyy      RU  50.7214  136.6319        57           
58     susuman      RU  62.7833  148.1667        38           
61    motygino      RU  58.1831   94.7592        56           


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
# Set parameters to search for a hotel
radius = 10000

params = { "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
aral - nearest hotel: No hotel found
udachny - nearest hotel: Вилюй
solnechnyy - nearest hotel: Заря
susuman - nearest hotel: No hotel found
motygino - nearest hotel: No hotel found
east london - nearest hotel: No hotel found
aykhal - nearest hotel: Уют
linxia chengguanzhen - nearest hotel: 如家酒店
dunedin - nearest hotel: Scenic Hotel Dunedin City
mount gambier - nearest hotel: The Old Mount Gambier Gaol
mount isa - nearest hotel: Ibis Styles
saynshand - nearest hotel: No hotel found
kurumkan - nearest hotel: Туяа
nyakabindi - nearest hotel: Peace hotel Maswa
vrede - nearest hotel: No hotel found
likasi - nearest hotel: Park Side
laojunmiao - nearest hotel: No hotel found
el bauga - nearest hotel: No hotel found
port elizabeth - nearest hotel: Waterford Hotel
shalqar - nearest hotel: Бекнур
bilibino - nearest hotel: No hotel found
sinop - nearest hotel: No hotel found
port sudan - nearest hotel: Ranya Hotel
orumiyeh - nearest hotel: هتل لاله
badiyah - nearest hotel:

,City,Country,Lat,Lng,Humidity,Hotel Name
1,aral,CN,40.5156,81.2636,36,No hotel found
9,udachny,RU,66.4167,112.4000,44,Вилюй
25,solnechnyy,RU,50.7214,136.6319,57,Заря
58,susuman,RU,62.7833,148.1667,38,No hotel found
61,motygino,RU,58.1831,94.7592,56,No hotel found
...,...,...,...,...,...,...
555,tyuratam,KZ,45.6507,63.3156,33,Спутник
556,zhanibek,KZ,49.4167,46.8500,33,No hotel found
560,plastun,RU,44.7561,136.2939,54,Россияночка
568,tagta,TM,41.6504,59.9164,30,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:

%%capture --no-display
# Configure the map plot

map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat', geo=True, tiles='OSM', size='Humidity', 
    color='City', hover_cols=['City', 'Humidity', 'Country', 'Hotel Name'],
    title='City Humidity Map'
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)